In [1]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import tree
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, HTML

/var/folders/k7/38v84p457m70dh2vksct50j80000gn/T/ipykernel_74699/4262510003.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [55]:
from datetime import datetime

def to_minutes(x):
    h,m = x.split(':')
    return int(h)*60 + int(m)

def findDayOfWeek(dateStr):
    date_object = datetime.strptime(dateStr, '%m/%d/%Y')
    dayOfWeek = date_object.strftime('%A')
    return dayOfWeek 

# findDayOfWeek('01/01/2006')

def flightNoStr(x):
    return str(int(x))

def classifyDelay(delay):
    if delay < -5:
        return 0 #flight is early
    elif delay > 5:
        return 1 #flight is delayed
    else:
        return 2 #flight is ontime
# def calculateDepTime()


In [56]:
# jetblue_B6_116_dep_time_range = (735, 885) #13:25= 13*60+25 - 70 min buffer or 13:35 = 13*60+35 + 70 minute buffer
RawDataProcessing = True

In [57]:
import os
# combinned_df = pd.DataFrame()
if RawDataProcessing:
    dataDir = 'data/arrivals/'
    fileNames = os.listdir(dataDir)
    dfList = []
    for file in fileNames:
        filePath = dataDir + file
        print('Processing file {}'.format(filePath))
        raw_df = pd.read_csv(filePath)
        # raw_df.head()
    
        airline_df = pd.DataFrame()
        #dropping flights other than MCO, ORD, and JFK 
        # to keep additional filters (jetblue['Origin Airport'] == 'MCO') | (jetblue['Origin Airport'] == 'ORD') |
        filtered_data  = raw_df[(raw_df['Origin Airport'] == 'MCO') | (raw_df['Origin Airport'] == 'JFK') | (raw_df['Origin Airport'] == 'ORD')].copy()
        # filtered_data['Origin Airport'].unique()
        airline_df['DATE'] = filtered_data['Date (MM/DD/YYYY)']
        airline_df['DAY'] = [findDayOfWeek(date) for date in filtered_data['Date (MM/DD/YYYY)']]
        
        filtered_data['Flight Number'] = [flightNoStr(flightNo) for flightNo in filtered_data['Flight Number']]
        filtered_data['Flight No'] = filtered_data['Carrier Code'].str.cat(filtered_data['Flight Number'], sep=' ')
        airline_df['FLIGHT NUMBER'] = filtered_data['Flight No']
    
        airline_df['ORIGIN'] = filtered_data['Origin Airport']
        
        #converting time data to minutes
        filtered_data['Scheduled Arrival Time'] = [to_minutes(s) for s in filtered_data['Scheduled Arrival Time']] #to_minutes(step_1['Scheduled Arrival Time'])
        filtered_data['Actual Arrival Time'] = [to_minutes(s) for s in filtered_data['Actual Arrival Time']]
        filtered_data['Wheels-on Time'] = [to_minutes(s) for s in filtered_data['Wheels-on Time']]
    
        filtered_data['Scheduled Departure Time'] = filtered_data['Scheduled Arrival Time'] - filtered_data['Scheduled Elapsed Time (Minutes)']
        airline_df['DEPARTURE TIME'] = filtered_data['Scheduled Departure Time']
        airline_df['ARRIVAL TIME'] = filtered_data['Scheduled Arrival Time']
        # jetblue_df
        airline_df = airline_df[airline_df['ARRIVAL TIME']>=0] ##dropping data where flight departed one earlier date
        # filtered_data['Arrival Delay (Minutes)']
        airline_df['ARRIVAL STATUS'] = [classifyDelay(delay) for delay in filtered_data['Arrival Delay (Minutes)']]
    
        # airline_df
        dfList.append(airline_df)
        
        # break
        # raw_data = pd.read_csv(file)
    
    combined_df=pd.concat(dfList, ignore_index=True)
    combined_df.to_csv('data/combined_arrival_data.csv')


Processing file data/arrivals/JetBlue_Airlines.csv
Processing file data/arrivals/Endeavor_Air.csv
Processing file data/arrivals/Southwest_Airlines.csv
Processing file data/arrivals/United_Airlines.csv
Processing file data/arrivals/Skywest_Airline.csv
Processing file data/arrivals/Republic_Airline.csv
Processing file data/arrivals/American_Airlines.csv
Processing file data/arrivals/Delta_Airlines.csv


In [58]:
if RawDataProcessing:
    dates = combined_df['DATE'].unique()
    # finalData =  pd.DataFrame()
    # MCO_flightList = []
    # JFK_flightList = []
    # ORD_flightList = []
    totalFlightList = []
    totalFlights = 0
    for date in dates:
        flightList_MCO = combined_df[(combined_df['DATE'] == date) & (combined_df['ORIGIN'] == 'MCO')]
        sortedList_MCO = flightList_MCO.sort_values(by='DEPARTURE TIME', ascending=True)
        mco_f = sortedList_MCO.shape[0]
        # print('{} flights on date {} from MCO'.format(mco_f, date))
        # sortedList_MCO
        sortedList_MCO['previous_flight_status'] = sortedList_MCO['ARRIVAL STATUS'].shift(periods=1)
        # sortedList_MCO
        totalFlightList.append(sortedList_MCO)
    
        flightList_JFK = combined_df[(combined_df['DATE'] == date) & (combined_df['ORIGIN'] == 'JFK')]
        sortedList_JFK = flightList_JFK.sort_values(by='DEPARTURE TIME', ascending=True)
        jfk_f = sortedList_JFK.shape[0]
        # print('{} flights on date {} from JFK'.format(jfk_f, date))
        # sortedList_JFK
        sortedList_JFK['previous_flight_status'] = sortedList_JFK['ARRIVAL STATUS'].shift(periods=1)
        # sortedList_JFK
        totalFlightList.append(sortedList_JFK)
    
        flightList_ORD = combined_df[(combined_df['DATE'] == date) & (combined_df['ORIGIN'] == 'ORD')]
        sortedList_ORD = flightList_ORD.sort_values(by='DEPARTURE TIME', ascending=True)
        ord_f = sortedList_ORD.shape[0]
        # print('{} flights on date {} from ORD'.format(ord_f, date))
        # sortedList_ORD
        sortedList_ORD['previous_flight_status'] = sortedList_ORD['ARRIVAL STATUS'].shift(periods=1)
        # sortedList_ORD
        totalFlightList.append(sortedList_ORD)
        flightsOnDay = mco_f + jfk_f + ord_f
        # print('{} flights on date {}'.format(flightsOnDay, date))
        totalFlights = totalFlights + flightsOnDay
        # if i==5:
        #     break
        # i+=1
    print('{} flights in total'.format(totalFlights))
    finalData = pd.concat (totalFlightList, ignore_index=True) 
    finalData.fillna(0, inplace=True)
    finalData['DEPARTURE TIME'] = finalData['DEPARTURE TIME'].astype(int)
    finalData['previous_flight_status'] = finalData['previous_flight_status'].astype(int)
    finalData.to_csv('data/combined_data_with_prev_flight_status.csv', index=False)
        # finalData = pd.concat([finalData, sortedList], ignore_index=True)
    # finalData

9336 flights in total


In [59]:
flight_data = pd.read_csv('data/combined_data_with_prev_flight_status.csv')
flight_data.head()
# flight_data.columns
# flight_data['FLIGHT NUMBER'].unique().shape
flight_data.isna().sum()

,DATE,DAY,FLIGHT NUMBER,ORIGIN,DEPARTURE TIME,ARRIVAL TIME,ARRIVAL STATUS,previous_flight_status
0,03/01/2006,Wednesday,B6 68,JFK,590,655,2,0
1,03/01/2006,Wednesday,B6 72,JFK,920,990,1,2
2,03/01/2006,Wednesday,B6 22,JFK,1375,1439,2,1
3,03/01/2006,Wednesday,OO 6844,ORD,600,706,0,0
4,03/01/2006,Wednesday,OO 5802,ORD,1330,1432,0,0


DATE                      0
DAY                       0
FLIGHT NUMBER             0
ORIGIN                    0
DEPARTURE TIME            0
ARRIVAL TIME              0
ARRIVAL STATUS            0
previous_flight_status    0
dtype: int64

In [100]:
#Building XGBoost model using the finalData
data_with_cat = flight_data.drop(columns = ['DATE', 'FLIGHT NUMBER'])
encoded_data = pd.get_dummies(data_with_cat, drop_first=True)
encoded_data.head()

,DEPARTURE TIME,ARRIVAL TIME,ARRIVAL STATUS,previous_flight_status,DAY_Monday,DAY_Saturday,DAY_Sunday,DAY_Thursday,DAY_Tuesday,DAY_Wednesday,ORIGIN_MCO,ORIGIN_ORD
0,590,655,2,0,False,False,False,False,False,True,False,False
1,920,990,1,2,False,False,False,False,False,True,False,False
2,1375,1439,2,1,False,False,False,False,False,True,False,False
3,600,706,0,0,False,False,False,False,False,True,False,True
4,1330,1432,0,0,False,False,False,False,False,True,False,True


In [101]:
X_train, X_test, y_train, y_test = train_test_split(encoded_data.drop(columns=['ARRIVAL STATUS']), encoded_data['ARRIVAL STATUS'], stratify = encoded_data['ARRIVAL STATUS'], test_size=0.1, random_state=42)
X_train.shape
X_test.shape

(8402, 11)

(934, 11)

In [120]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

total_Data_scaled = pd.DataFrame(scaler.fit_transform(encoded_data), columns=encoded_data.columns, index=encoded_data.index)
total_y = encoded_data['ARRIVAL STATUS']
# X_test_scaled

In [116]:
# X_train_scaled.rename({'DEPARTURE TIME':'DEPARTURE_TIME', 'ARRIVAL TIME':'ARRIVAL_TIME'})
# X_test_scaled.rename({'DEPARTURE TIME':'DEPARTURE_TIME', 'ARRIVAL TIME':'ARRIVAL_TIME'})

# dtrain_class = xgb.DMatrix(X_train_scaled, y_train, enable_categorical=True)
# dtest_class = xgb.DMatrix(X_test_scaled, y_test, enable_categorical=True)



In [117]:
def accuracy_score(test_output):
    total_count = len(test_output)
    wrong_count = len(test_output[test_output['Predicted Arrival Status'] != test_output['ARRIVAL STATUS']])
    accuracy = wrong_count/total_count
    return accuracy

In [118]:
tree_methods = ['exact', 'hist', 'approx']
tree_depth = [4, 6, 7, 8, 10, 15]
learning_rates = [0.05, 0.1, 0.2, 0.3, 0.4]

best_accuracy = 0
best_params = {}

In [119]:
for method in tree_methods:
    for depth in tree_depth:
        for lr in learning_rates:
            params = {
                "objective": "reg:squarederror",
                "tree_method": method,
                "max_depth" : depth,
                "learning_rate" : lr,
                "n_estimators": 200
            }
            xgbModel = xgb.XGBClassifier(**params)
            # xgbModel = GradientBoostingClassifier()
            # xgbModel = GradientBoostingClassifier()
            xgbModel.fit(X_train_scaled, y_train)
            y_pred = pd.DataFrame(xgbModel.predict(X_test_scaled), columns=['Predicted Arrival Status'], index=X_test.index)
            
            test_output = y_pred.merge(y_test, left_index=True, right_index=True)
            # test_output.head()
            accuracy = accuracy_score(test_output)
            print('Accuracy of the xgb model is {}'.format(accuracy))
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = params

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

Accuracy of the xgb model is 0.4860813704496788


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

Accuracy of the xgb model is 0.49143468950749464


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

...

Accuracy of the xgb model is 0.5856531049250535


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

Accuracy of the xgb model is 0.5877944325481799


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.4, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

Accuracy of the xgb model is 0.5920770877944326


In [110]:
print('Best Accuracy using xgb model is {}'.format(best_accuracy))
print('Best Parameter using xgb model is {}'.format(best_params))

Best Accuracy using xgb model is 0.5942184154175589
Best Parameter using xgb model is {'objective': 'reg:squarederror', 'tree_method': 'approx', 'max_depth': 10, 'learning_rate': 0.4, 'n_estimators': 200}


In [ ]:
best_model = xgb.XGBClassifier(**best_params)

best_model.fit(total_Data_scaled, total_y)




y_pred = pd.DataFrame(xgbModel.predict(X_test_scaled), columns=['Predicted Arrival Status'], index=X_test.index)

test_output = y_pred.merge(y_test, left_index=True, right_index=True)
# test_output.head()
accuracy = accuracy_score(test_output)
print('Accuracy of the xgb model is {}'.format(accuracy))
if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_params = params

,Predicted Arrival Status,ARRIVAL STATUS
8698,0.98,1
4996,0.72,1
9129,0.78,2
4428,0.92,0
2052,1.11,0


Accuracy of the default xgb model is 1.0


,Predicted Arrival Status,ARRIVAL STATUS
8698,0.98,1
4996,0.73,1
9129,0.78,2
4428,1.01,0
2052,1.21,0


Accuracy of the default xgb model is 1.0


,Predicted Arrival Status,ARRIVAL STATUS
8698,1.19,1
4996,0.78,1
9129,0.75,2
4428,1.04,0
2052,1.26,0


...

Accuracy of the default xgb model is 1.0


,Predicted Arrival Status,ARRIVAL STATUS
8698,1.02,1
4996,1.20,1
9129,1.33,2
4428,1.00,0
2052,1.02,0


Accuracy of the default xgb model is 1.0


,Predicted Arrival Status,ARRIVAL STATUS
8698,1.01,1
4996,1.20,1
9129,1.34,2
4428,1.00,0
2052,1.00,0


Accuracy of the default xgb model is 1.0
